# Sprint 11: Proyecto Integrado 2

## Índice
-   [Índice](#índice)
-   [Descripción del proyecto](#descripción-del-proyecto)
-   [Paso 1. Descargar los datos](#paso-1-descargar-los-datos)
-   [Paso 2. Llevar a cabo el análisis exploratorio de datos (EDA)](#paso-2-llevar-a-cabo-el-análisis-exploratorio-de-datos-eda)}
-   [Paso 3. Construir un modelo para predecir la cancelación de usuarios](#paso-3-construir-un-modelo-para-predecir-la-cancelación-de-usuarios)
-   [Paso 4. Crear clústeres de usuarios/as](#paso-4-crear-clústeres-de-usuariosas)
-   [Paso 5. Saca conclusiones y haz recomendaciones básicas sobre el trabajo con clientes](#paso-5-saca-conclusiones-y-haz-recomendaciones-básicas-sobre-el-trabajo-con-clientes)

## Descripción del proyecto

Descripción del proyecto
La cadena de gimnasios Model Fitness está desarrollando una estrategia de interacción con clientes basada en datos analíticos.

Uno de los problemas más comunes que enfrentan los gimnasios y otros servicios es la pérdida de clientes. ¿Cómo descubres si un/a cliente ya no está contigo? Puedes calcular la pérdida en función de las personas que se deshacen de sus cuentas o no renuevan sus contratos. Sin embargo, a veces no es obvio que un/a cliente se haya ido: puede que se vaya de puntillas.

Los indicadores de pérdida varían de un campo a otro. Si un usuario o una usuaria compra en una tienda en línea con poca frecuencia, pero con regularidad, no se puede decir que ha huido. Pero si durante dos semanas no ha abierto un canal que se actualiza a diario, es motivo de preocupación: es posible que tu seguidor o seguidor/a se haya aburrido y te haya abandonado.

En el caso de un gimnasio, tiene sentido decir que un/a cliente se ha ido si no viene durante un mes. Por supuesto, es posible que estén en Cancún y retomen sus visitas cuando regresen, pero ese no es un caso típico. Por lo general, si un/a cliente se une, viene varias veces y luego desaparece, es poco probable que regrese.

Con el fin de combatir la cancelación, Model Fitness ha digitalizado varios de sus perfiles de clientes. Tu tarea consiste en analizarlos y elaborar una estrategia de retención de clientes.

Tienes que:

-   Aprender a predecir la probabilidad de pérdida (para el próximo mes) para cada cliente.
-   Elaborar retratos de usuarios típicos: selecciona los grupos más destacados y describe sus características principales.
-   Analizar los factores que más impactan la pérdida.
-   Sacar conclusiones básicas y elaborar recomendaciones para mejorar la atención al cliente:
    -   identificar a los grupos objetivo;
    -   sugerir medidas para reducir la rotación;
    -   describir cualquier otro patrón que observes con respecto a la interacción con los clientes.

## Paso 1. Descargar los datos

Model Fitness te aportó archivos CSV que contienen los datos sobre la cancelación de un mes en concreto e información del mes que lo precedía. El dataset incluye los siguientes campos:

-   'Churn' — la cancelación para el mes en cuestión
-   Campos de dataset actuales:
    -   Datos del usuario del mes anterior
        -   'gender'.
        -   'Near_Location' — si el/la usuario/a vive o trabaja en el vecindario donde se encuentra el gimnasio.
        -   'Partner' — si el/la usuario/a trabaja en una compañía asociada (el gimnasio tiene empresas asociadas cuyos empleados obtienen descuentos; en esos casos el gimnasio almacena información sobre los empleadores de los clientes).
        -   Promo_friends — si el/la usuario/a originalmente se inscribió mediante una oferta “trae a un/a amigo/a” (se utilizó el código promocional de un/a amigo/a cuando pagaron el primer abono).
        -   'Phone' — si el/la usuario/a aportó el número de teléfono.
        -   'Age'.
        -   'Lifetime' — el tiempo (en meses) desde que el/la usuario/a llegó por primera vez al gimnasio.
-   Datos del registro de visitas y compras y datos sobre el estado actual de la membresía:
    -   'Contract_period' — 1 mes, 3 meses, 6 meses o 1 año.
    -   'Month_to_end_contract' — los meses que faltan hasta que expire el contrato.
    -   'Group_visits' — si el/la usuario/a participa en sesiones grupales.
    -   'Avg_class_frequency_total' — frecuencia media de visitas por semana a lo largo de la vida del cliente.
    -   'Avg_class_frequency_current_month' — frecuencia media de visitas por semana durante el mes en curso.
    -   'Avg_additional_charges_total' — cantidad total de dinero gastado en otros servicios del gimnasio: cafetería, productos deportivos, cosméticos, masajes, etc.

<b> Ruta de archivo: /datasets/gym_churn_us.csv. :</b> https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gym_churn_us.csv

## Paso 2. Llevar a cabo el análisis exploratorio de datos (EDA)

-   Observa el dataset: ¿contiene alguna característica ausente? Estudia los valores promedio y la desviación estándar (utiliza el método describe()).
-   Observa los valores medios de las características en dos grupos: para las personas que se fueron (cancelación) y para las que se quedaron (utiliza el método groupby()).
-   Traza histogramas de barras y distribuciones de características para aquellas personas que se fueron (cancelación) y para las que se quedaron.
-   Crea una matriz de correlación y muéstrala.

In [1]:
# Cargar todas las librerías
from scipy import stats as st
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
import pandas as pd
import math as mt
# from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import os
from plotly import graph_objects as go

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans

In [3]:
# La función data_load() carga un archivo CSV llamado 'logs_exp_us.csv' desde una carpeta llamada 'datasets', 
# donde los valores en el archivo están separados por tabulaciones, y devuelve un DataFrame con los datos.

def data_load():	
    ruta_archivo = os.path.join('datasets', 'gym_churn_us.csv')
    data = pd.read_csv(
	    ruta_archivo#, sep= '\t'
	)
    return data

In [ ]:
# Se llama a la función data_loas() y se guarda el datarame resultante en 'data'
data = data_load()
data.columns = data.columns.str.lower()
data.info()

In [ ]:
for feature in data.columns:
  print(feature)
  print(data[feature].head(5).unique())

In [6]:
data['month_to_end_contract'] = data['month_to_end_contract'].astype(int)

In [ ]:
data.info()

In [8]:
# El código revisa si el DataFrame data contiene filas duplicadas. Si es así,
# elimina las duplicadas utilizando el método drop_duplicates().

if len(data[data.duplicated()]) > 0:
    data = data.drop_duplicates()

In [ ]:
data.head()

In [ ]:
data.describe()

In [11]:
data_gp = data.groupby(by= 'churn').mean()

In [ ]:
data_gp

In [ ]:
for feature in data.columns:
  sns.distplot(data[feature], bins = 12)
  plt.show()

In [14]:
cm = data.corr() 

In [ ]:
cm

## Paso 3. Construir un modelo para predecir la cancelación de usuarios

Crea un modelo de clasificación binaria para clientes donde la característica objetivo es la marcha del usuario o la usuaria el mes siguiente.

-   Divide los datos en conjuntos de entrenamiento y validación utilizando la función `train_test_split()`.
-   Entrena el modelo en el set de entrenamiento con dos métodos:
    -   regresión logística;
    -   bosque aleatorio.
-   Evalúa la exactitud, precisión y recall para ambos modelos utilizando los datos de validación. Utilízalos para comparar los modelos. ¿Qué modelo dio mejores resultados?

Recuerda indicar el parámetro `random_state` cuando dividas los datos y definas el algoritmo.

In [16]:
# define la función que podrá generar nuestras métricas
def print_all_metrics(y_true, y_pred, y_proba, title = 'Métricas de clasificación'):
    print(title)
    print('\tAccuracy: {:.2f}'.format(accuracy_score(y_true, y_pred)))
    print('\tPrecision: {:.2f}'.format(precision_score(y_true, y_pred)))
    print('\tRecall: {:.2f}'.format(recall_score(y_true, y_pred)))
    print('\tF1 score: {:.2f}'.format(f1_score(y_true, y_pred)))
    print('\tROC_AUC: {:.2f}'.format(roc_auc_score(y_true, y_proba)))

In [17]:
# divide los datos en características (la matriz X) y una variable objetivo (y)
X = data.drop(['churn', 'month_to_end_contract'], axis=1)
y = data['churn']

# divide los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [18]:
# estandariza los datos utilizando el método StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_st = scaler.transform(X_train)
X_test_st = scaler.transform(X_test)

In [ ]:
# define el algoritmo para el modelo de regresión logística
lr_model = LogisticRegression(random_state=0, max_iter=1000)
# entrena el modelo 
lr_model.fit(X_train_st, y_train)
# utiliza el modelo entrenado para hacer previsiones
lr_predictions = lr_model.predict(X_test_st)
lr_probabilities = lr_model.predict_proba(X_test_st)[:,1]
# muestra todas las métricas
print_all_metrics(y_test, lr_predictions, lr_probabilities , title='Métricas de regresión logística:')

In [ ]:
# define el algoritmo para el nuevo modelo de random forest
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 0)
# entrena el modelo de random forest
rf_model.fit(X_train_st, y_train)
# utiliza el modelo entrenado para hacer previsiones
rf_predictions = rf_model.predict(X_test_st)
rf_probabilities = rf_model.predict_proba(X_test_st)[:,1]
# muestra todas las métricas
print_all_metrics(y_test, rf_predictions, rf_probabilities, title='Métricas de random forest classifier:')

Al comparar los resultados de los dos modelos de clasificación binaria (Regresión Logística y Bosque Aleatorio) con base en las métricas proporcionadas, podemos sacar las siguientes conclusiones:

### 1. **Accuracy (Exactitud):**
- **Regresión Logística:** 0.92
- **Bosque Aleatorio:** 0.91

La exactitud de ambos modelos es muy similar, con una ligera ventaja para la regresión logística. Sin embargo, la diferencia es pequeña, lo que indica que ambos modelos tienen un rendimiento global bastante similar en términos de predicción correcta de clases.

### 2. **Precision (Precisión):**
- **Regresión Logística:** 0.85
- **Bosque Aleatorio:** 0.83

La precisión mide cuántos de los ejemplos clasificados como positivos son realmente positivos. Aquí, la regresión logística tiene una mejor precisión, lo que significa que es ligeramente más efectiva para evitar falsos positivos. Esto puede ser importante en situaciones donde los falsos positivos tienen un impacto significativo.

### 3. **Recall (Sensibilidad):**
- **Regresión Logística:** 0.83
- **Bosque Aleatorio:** 0.81

El recall mide cuántos de los ejemplos positivos reales fueron clasificados correctamente como positivos. Nuevamente, la regresión logística tiene una pequeña ventaja sobre el bosque aleatorio, lo que indica que es un poco mejor para capturar los verdaderos positivos. Esto es relevante cuando es crucial detectar la mayor cantidad posible de casos positivos (como en la detección de fraudes o enfermedades).

### 4. **F1 Score:**
- **Regresión Logística:** 0.84
- **Bosque Aleatorio:** 0.82

El F1 score es la media armónica entre precisión y recall, por lo que representa un equilibrio entre ambas métricas. La regresión logística tiene un F1 score ligeramente superior, lo que indica que en términos de un balance general entre precisión y recall, se comporta un poco mejor que el bosque aleatorio.

### 5. **ROC_AUC (Área bajo la curva ROC):**
- **Regresión Logística:** 0.97
- **Bosque Aleatorio:** 0.97

Ambos modelos tienen un valor de ROC_AUC idéntico, lo que indica que ambos tienen un poder discriminativo excelente. Esto significa que los modelos son igualmente efectivos para separar las clases positivas de las negativas. Un valor de 0.97 en ROC_AUC es muy bueno, lo que sugiere que ambos modelos son altamente eficientes en esta tarea.


### **Conclusiones:**

1. **Regresión Logística frente a Bosque Aleatorio:**
   - **Rendimiento General:** Ambos modelos tienen rendimientos bastante similares en todas las métricas, aunque la regresión logística muestra una ligera ventaja en precisión, recall y F1 score. Esto sugiere que, si tu objetivo es maximizar estas métricas en este conjunto de datos, la regresión logística podría ser preferible.
   - **ROC_AUC:** Dado que ambos modelos tienen el mismo valor de ROC_AUC (0.97), ambos tienen un excelente poder para distinguir entre las dos clases, lo cual es fundamental en tareas de clasificación binaria.

2. **Uso de los Modelos según el Contexto:**
   - **Regresión Logística:** Es preferible si te interesa un balance entre precisión y recall (dado el mayor F1 score) y si necesitas un modelo más interpretable, ya que la regresión logística ofrece coeficientes que permiten interpretar la relación entre las variables independientes y el resultado.
   - **Bosque Aleatorio:** Aunque en este caso es ligeramente inferior, sigue siendo una buena opción. Puede ser más útil en problemas donde el modelo debe lidiar con datos no lineales o con muchas características interdependientes, debido a su capacidad para capturar interacciones complejas entre las variables.

En resumen, la **Regresión Logística** ofrece un rendimiento ligeramente superior en este caso particular, pero **ambos modelos son efectivos** y su elección dependerá del contexto y de las prioridades (precisión, interpretabilidad, capacidad para manejar datos complejos).

## Paso 4. Crear clústeres de usuarios/as

Deja de lado la columna con datos sobre la cancelación e identifica los clústeres de objetos (usuarios/as):

-   Estandariza los datos.
-   Utiliza la función `linkage()` para crear una matriz de distancias basada en la matriz de características estandarizada y trazar un dendrograma. Nota: ¡renderizar el dendrograma puede llevar tiempo! Utiliza el gráfico resultante para estimar el número de clústeres que puedes destacar.
-   Entrena el modelo de clustering con el algortimo K-means y predice los clústeres de clientes. (Deja que el número de clústeres sea `n=5` para que sea más sencillo comparar los resultados con los del resto del estudiantado. Sin embargo, en la vida real, nadie te dará tales pistas, así que tendrás que decidir basándote en el gráfico del paso anterior.)
-   Mira los valores medios de característica para los clústeres.¿Hay algo que te llame la atención?
-   Traza distribuciones de características para los clústeres. ¿Notas algo?
-   Calcula la tasa de cancelación para cada clúster (utiliza el método groupby()). ¿Difieren en términos de tasa de cancelación? ¿Qué grupos son propensos a irse y cuáles son leales?

In [21]:
# estandarización obligatoria de los datos antes de pasarlo al algoritmo
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [ ]:


linked = linkage(X_sc, method = 'ward')

plt.figure(figsize=(15, 10))  
dendrogram(linked, orientation='top')
plt.title('Agrupación jerárquica para GYM')
plt.show()

In [23]:
# define el modelo k_means con 5 clústeres
km = KMeans(n_clusters=5)
# predice los clústeres para las observaciones (el algoritmo les asigna un número de 0 a 4)
labels = km.fit_predict(X_sc)

In [ ]:
# almacena etiquetas de clúster en el campo de nuestro conjunto de datos
data['cluster_km'] = labels

# obtiene las estadísticas de los valores medios de las características por clúster
data.groupby(['cluster_km']).mean()

In [41]:
# define la función para representar gráficos de características pareadas para los clústeres
def show_clusters_on_plot(df, x_name, y_name, cluster_name):
    plt.figure(figsize=(5,5))
    sns.scatterplot(x=df[x_name], y=df[y_name], 
                    hue=df[cluster_name], palette='Paired')
    plt.title('{} vs {}'.format(x_name, y_name))
    plt.show()

In [ ]:
data.info()

In [69]:
columns = ['gender', 'near_location', 'partner', 'promo_friends', 'phone', 'contract_period', 'group_visits', 'age', 'avg_additional_charges_total', 'month_to_end_contract',
            'lifetime', 'avg_class_frequency_total', 'avg_class_frequency_current_month', 'churn']

In [ ]:
# for col1 in columns:
#     for col2 in columns:
#         if col1 != col2:
#             show_clusters_on_plot(data, col1, col2, 'cluster_km')

Una vez analizadas las distribuciones de características para los clústeres se redujeron a las combinaciones a analizar a las listadas a continuación, ya que fueron en las que e encontraron patrones y distribuciones más interesantes y que permitían visualizar sus relaciones.

In [ ]:
selected_columns = ['avg_additional_charges_total', 'age',  'avg_class_frequency_total', 'avg_class_frequency_current_month']

In [ ]:
for col1 in selected_columns:
    for col2 in selected_columns:
        if col1 != col2:
            show_clusters_on_plot(data, col1, col2, 'cluster_km')

In [ ]:
churn_rate = data.groupby(['cluster_km'], as_index= False)['churn'].mean()
churn_rate['churn'] = churn_rate['churn']*100
# churn_rate.info()
cluster = 0
for rate in churn_rate['churn']:
    print('Cluster ', cluster, 'churn rate:')
    print('Rate:{:.2f}'.format(rate), '%\n')
    cluster += 1


### Descripción por clúster:

1. **Clúster 0**:
   - **Género**: Todos son hombres (`gender = 1.0`).
   - **Ubicación cercana (near_location)**: Todos viven cerca (`near_location = 1.0`).
   - **Churn (deserción)**: Alto (`churn = 0.450882`), es decir, el 45% de los clientes en este clúster han abandonado.
   - **Contrato**: Contrato corto (`contract_period = 2.81 meses`), lo que sugiere que la mayoría tiene membresías a corto plazo.
   - **Visitas grupales**: Moderadamente bajas (`group_visits = 0.376`).
   - **Frecuencia de clases**: Baja frecuencia general de clases y en el mes actual (`avg_class_frequency_total = 1.42`, `avg_class_frequency_current_month = 1.22`).
   - **Conclusión**: Este grupo tiene una alta tasa de deserción, contratos cortos y baja frecuencia de uso. Pueden estar menos comprometidos con el gimnasio.

2. **Clúster 1**:
   - **Género**: Aproximadamente mitad hombres, mitad mujeres (`gender = 0.49`).
   - **Ubicación cercana**: La mayoría vive cerca (`near_location = 0.995`).
   - **Churn (deserción)**: Muy bajo (`churn = 0.119`), solo el 12% ha abandonado.
   - **Contrato**: Contratos largos (`contract_period = 7.83 meses`).
   - **Frecuencia de clases**: Relativamente alta frecuencia de clases tanto en total como en el mes actual (`avg_class_frequency_total = 1.67`, `avg_class_frequency_current_month = 1.62`).
   - **Conclusión**: Este grupo parece estar muy comprometido con el gimnasio, con contratos largos, alta frecuencia de visitas y una baja tasa de abandono.

3. **Clúster 2**:
   - **Género**: Todos son mujeres (`gender = 0.0`).
   - **Ubicación cercana**: Todos viven cerca (`near_location = 1.0`).
   - **Churn (deserción)**: Moderadamente alto (`churn = 0.429`), similar al clúster 0.
   - **Contrato**: Contratos cortos (`contract_period = 2.71 meses`).
   - **Frecuencia de clases**: Baja frecuencia general de clases (`avg_class_frequency_total = 1.38`).
   - **Conclusión**: Similar al clúster 0, este grupo (que es exclusivamente mujeres) muestra una tasa de abandono relativamente alta, contratos cortos y baja frecuencia de visitas, lo que indica menor compromiso.

4. **Clúster 3**:
   - **Género**: Aproximadamente mitad hombres, mitad mujeres (`gender = 0.54`).
   - **Ubicación cercana**: La mayoría vive cerca (`near_location = 0.968`).
   - **Churn (deserción)**: Muy bajo (`churn = 0.012`), lo que sugiere una tasa de abandono extremadamente baja.
   - **Contrato**: Contratos largos (`contract_period = 5.81 meses`).
   - **Frecuencia de clases**: Alta frecuencia de clases en total y en el mes actual (`avg_class_frequency_total = 3.07`, `avg_class_frequency_current_month = 3.08`).
   - **Conclusión**: Este es el grupo más comprometido con el gimnasio, con alta frecuencia de uso y una tasa de abandono extremadamente baja. También tienen contratos relativamente largos.

5. **Clúster 4**:
   - **Género**: Aproximadamente mitad hombres, mitad mujeres (`gender = 0.49`).
   - **Ubicación cercana**: Ninguno vive cerca (`near_location = 0.0`), lo que es una diferencia notable respecto a los otros grupos.
   - **Churn (deserción)**: Moderadamente alto (`churn = 0.417`), similar a los clústeres 0 y 2.
   - **Contrato**: Contratos cortos (`contract_period = 2.88 meses`).
   - **Frecuencia de clases**: Moderada (`avg_class_frequency_total = 1.68`), mejor que los clústeres 0 y 2, pero menor que los clústeres 1 y 3.
   - **Conclusión**: Este grupo es interesante porque, aunque tiene una tasa de abandono moderadamente alta, se distingue por el hecho de que no viven cerca del gimnasio, lo que podría estar afectando su comportamiento y compromiso.

---

### **Conclusiones generales**:

1. **Compromiso con el gimnasio**:
   - Los **clústeres 1 y 3** muestran el mayor compromiso con el gimnasio. Tienen bajas tasas de abandono, contratos largos y frecuencias de visitas altas. En particular, el clúster 3 es el más comprometido de todos, con una tasa de abandono casi nula.
   - Los **clústeres 0, 2 y 4** presentan tasas de deserción más altas y menores frecuencias de visitas, lo que sugiere un menor compromiso. Estos clústeres podrían ser el foco de estrategias de retención.

2. **Efecto de la ubicación cercana**:
   - La mayoría de los clientes de los clústeres viven cerca del gimnasio, excepto el **clúster 4**, donde ninguno vive cerca. Esto podría estar contribuyendo a la tasa de deserción más alta en este grupo. La distancia al gimnasio parece ser un factor clave en la retención.

3. **Duración del contrato**:
   - Los clústeres con contratos más largos (como los clústeres 1 y 3) tienden a tener tasas de deserción más bajas. Esto sugiere que ofrecer contratos más largos podría ser una estrategia efectiva para mejorar la retención de los clientes.

4. **Segmentación por género**:
   - Los **clústeres 0 y 2** parecen estar segregados por género (0 es exclusivamente masculino y 2 exclusivamente femenino), pero tienen características similares, incluyendo tasas de deserción más altas y contratos cortos. El género, en estos casos, no parece ser un diferenciador importante, ya que ambos muestran comportamientos similares.

### **Recomendaciones**:

1. **Intervenciones para retener a los clústeres 0, 2 y 4**: Estos grupos presentan tasas de deserción más altas y contratos más cortos. Las intervenciones podrían incluir:
   - Ofrecer incentivos para contratos más largos.
   - Programas de fidelización o promociones dirigidas a estos grupos.
   - Facilitar el acceso al gimnasio o crear un sentido de comunidad para mejorar el compromiso.

2. **Revisión del impacto de la ubicación para el clúster 4**: Este grupo se distingue por no vivir cerca del gimnasio. Sería interesante explorar cómo la distancia afecta su uso del gimnasio y si pueden implementarse soluciones, como entrenamientos en línea o ubicaciones adicionales.

3. **Refuerzo de las estrategias actuales para los clústeres 1 y 3**: Estos grupos ya están altamente comprometidos y muestran bajas tasas de abandono. Mantener los incentivos que los mantienen fieles es crucial, pero también se puede buscar nuevas formas de aumentar su lealtad a largo plazo.

## Paso 5. Saca conclusiones y haz recomendaciones básicas sobre el trabajo con clientes

Llega a conclusiones y formula recomendaciones con respecto a la estrategia para la interacción y retención de clientes.

No necesitas entrar en detalles. Bastarán tres o cuatro principios esenciales y ejemplos de su implementación en forma de pasos de marketing específicos.

### **Análisis General de Model Fitness**

Al observar los resultados de los dos análisis realizados (comparación de modelos de clasificación binaria y segmentación mediante K-Means), obtenemos información clave sobre el comportamiento de los clientes de **Model Fitness**. Estas conclusiones pueden ser valiosas para mejorar la atención y la retención de los miembros de la cadena de gimnasios.

### 1. **Comparación de Modelos de Clasificación Binaria**:
   - Se compararon dos modelos, **Regresión Logística** y **Bosque Aleatorio**, para predecir la deserción de clientes. Ambos modelos mostraron un rendimiento similar en términos de precisión y poder discriminativo, pero la **Regresión Logística** sobresalió ligeramente en métricas como precisión, recall y F1 score.
   - Ambos modelos tienen un **ROC_AUC** alto (0.97), lo que indica que ambos son eficaces para distinguir entre clientes que abandonan y los que permanecen.
   - **Recomendación**: **Model Fitness** debería implementar un modelo predictivo de deserción, utilizando la regresión logística como base, para identificar con precisión a los clientes en riesgo de abandonar. Esto permitirá enfocar recursos en intervenciones preventivas, como promociones personalizadas o programas de fidelización, dirigidos a los clientes más propensos a cancelar su membresía.

### 2. **Resultados del Análisis de Clusters (K-Means)**:
   El análisis K-Means segmentó a los clientes de **Model Fitness** en cinco clústeres distintos. Cada grupo tiene características particulares que indican diferentes niveles de compromiso y probabilidad de abandono.

   - **Clúster 1 y 3**: 
     - Son los clientes más comprometidos, con bajas tasas de deserción, contratos largos y alta frecuencia de uso.
     - **Recomendación**: Estos clientes son leales y valiosos. **Model Fitness** debería enfocarse en fortalecer su relación con ellos a través de programas exclusivos como membresías premium, recompensas por lealtad, o acceso a entrenadores personales o eventos especiales. Mantener su satisfacción ayudará a consolidar su lealtad a largo plazo.

   - **Clúster 0, 2 y 4**:
     - Estos grupos muestran tasas de deserción significativamente más altas y menor compromiso. Los contratos son más cortos y la frecuencia de uso es baja.
     - **Clúster 4** es particularmente interesante, ya que los clientes no viven cerca de ninguna ubicación de **Model Fitness**, lo cual podría estar afectando su compromiso.
     - **Recomendación**: **Model Fitness** debe realizar intervenciones específicas para retener a estos clientes, como:
       - **Clúster 0 y 2**: Ofrecer incentivos para contratos más largos (descuentos por renovar por más meses) y mejorar la comunicación para invitar a clases grupales, eventos o promociones que aumenten la frecuencia de uso.
       - **Clúster 4**: Considerar alternativas para clientes que viven lejos, como entrenamientos en línea, apps con programas personalizados o la creación de nuevas ubicaciones más cercanas.

### **Conclusiones Generales y Acciones Recomendadas para Mejorar la Atención al Cliente**:

1. **Mejora en la Retención de Clientes**:
   - Los **clústeres 1 y 3** de **Model Fitness** están muy comprometidos, pero la empresa debe seguir incentivando su lealtad. Implementar programas de **atención personalizada** para estos clientes valiosos, como planes de entrenamiento a medida y acceso prioritario a nuevas clases o instalaciones, puede aumentar aún más su satisfacción.
   - Para los **clústeres 0, 2 y 4**, que tienen un mayor riesgo de deserción, es importante ofrecer promociones específicas que los mantengan conectados al gimnasio. **Model Fitness** debería mejorar su seguimiento proactivo, contactando a estos clientes de manera regular y ofreciéndoles incentivos para aumentar su frecuencia de visitas y su permanencia en el gimnasio.

2. **Ajustes en la Estrategia de Contratos y Ubicación**:
   - El análisis muestra que los clientes con contratos más largos tienen menores tasas de abandono. **Model Fitness** debería incentivar a los clientes a suscribirse a planes más largos mediante descuentos o ventajas adicionales (como meses gratuitos o acceso a áreas premium).
   - **Ubicación y accesibilidad**: El clúster 4, cuyos miembros no viven cerca del gimnasio, sugiere que la distancia es un factor importante en la retención. **Model Fitness** puede ofrecer soluciones como **entrenamientos virtuales**, o incluso explorar nuevas ubicaciones o alianzas con gimnasios más cercanos para estos clientes.

3. **Personalización y Comunicación**:
   - **Model Fitness** debe desarrollar un enfoque más personalizado para mejorar la atención al cliente, basado en los datos. Utilizar la información del modelo predictivo de deserción y el análisis de clústeres permitirá diseñar estrategias específicas para cada grupo de clientes. Un enfoque personalizado puede incluir el envío de ofertas y recordatorios sobre las clases que más disfrutan, o el uso de apps móviles para fomentar la interacción continua.
   - Mejorar la **comunicación proactiva** con los clientes en riesgo puede ser clave para reducir la deserción. Esto incluye encuestas de satisfacción, llamadas o correos electrónicos personalizados para verificar su experiencia y ofrecer soluciones antes de que cancelen su membresía.

### **Conclusión Final**:
**Model Fitness** puede mejorar considerablemente la atención y retención de sus clientes mediante la implementación de modelos predictivos y segmentación avanzada. La personalización de las ofertas, la mejora de la accesibilidad para clientes distantes y el fortalecimiento de la relación con los clientes más fieles son pasos críticos para garantizar el éxito a largo plazo. Al adaptar sus estrategias a las necesidades específicas de cada clúster, **Model Fitness** puede optimizar la experiencia de sus usuarios y mantener una base de clientes más satisfecha y leal.